In [1]:
import numpy as np
from torchvision.models.resnet import resnet50
import torchvision.transforms as transforms
from PIL import Image
import torch

# Resize and normalize input

In [2]:
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# First test ResNet50 in PyTorch

In [3]:
net = resnet50(pretrained=True)
net.eval()
im = Image.open('dog.jpeg')
x = transform_test(im)
x = x.unsqueeze(dim=0)

![alt text](dog.jpeg "Dog test image")

# Save the input vector so that we can use it for OpenVino tests

In [4]:
print (x.shape)
np.save("test_in_vector", x)

!ls -l test_in_vector.npy

torch.Size([1, 3, 224, 224])
-rw-rw-r-- 1 ngeorgis ngeorgis 602240 Jan 26 13:17 test_in_vector.npy


# Run inference 

In [5]:
res = net(x)[0]
res.shape

torch.Size([1000])

In [6]:
print ("First ten output values: ", net(x)[0][0:10])

First ten output values:  tensor([ 0.8970, -3.0496, -2.7042, -0.7480, -3.5622, -1.7981, -4.8486,  0.1094,
         0.8685,  0.0536], grad_fn=<SliceBackward>)


In [7]:
values, indices = res.max(0)
print (indices)

tensor(235)


In [8]:
values, indices = torch.topk(res, 10)
print (values)
print (indices)

tensor([15.4252, 11.2401, 11.0313,  9.7304,  8.9736,  8.8621,  8.5262,  8.4578,
         7.9833,  7.8347], grad_fn=<TopkBackward>)
tensor([235, 255, 225, 174, 170, 224, 275, 160, 169, 227])


In [9]:
number_top = 10

In [10]:
labels ="test_model.labels"
if labels:
    with open(labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None

print (labels_map)

['n01440764_tench,_Tinca_tinca', 'n01443537_goldfish,_Carassius_auratus', 'n01484850_great_white_shark,_white_shark,_man-eater,_man-eating_shark,_Carcharodon_carcharias', 'n01491361_tiger_shark,_Galeocerdo_cuvieri', 'n01494475_hammerhead,_hammerhead_shark', 'n01496331_electric_ray,_crampfish,_numbfish,_torpedo', 'n01498041_stingray', 'n01514668_cock', 'n01514859_hen', 'n01518878_ostrich,_Struthio_camelus', 'n01530575_brambling,_Fringilla_montifringilla', 'n01531178_goldfinch,_Carduelis_carduelis', 'n01532829_house_finch,_linnet,_Carpodacus_mexicanus', 'n01534433_junco,_snowbird', 'n01537544_indigo_bunting,_indigo_finch,_indigo_bird,_Passerina_cyanea', 'n01558993_robin,_American_robin,_Turdus_migratorius', 'n01560419_bulbul', 'n01580077_jay', 'n01582220_magpie', 'n01592084_chickadee', 'n01601694_water_ouzel,_dipper', 'n01608432_kite', 'n01614925_bald_eagle,_American_eagle,_Haliaeetus_leucocephalus', 'n01616318_vulture', 'n01622779_great_grey_owl,_great_gray_owl,_Strix_nebulosa', 'n01629

In [11]:
print("Top {} results:\n===============\n".format(number_top))
for i, probs in enumerate(res):
    if (i<number_top):
        print( labels_map[indices[i]], "\t\t", values[i], indices[i])

Top 10 results:

n02106662_German_shepherd,_German_shepherd_dog,_German_police_dog,_alsatian 		 tensor(15.4252, grad_fn=<SelectBackward>) tensor(235)
n02111129_Leonberg 		 tensor(11.2401, grad_fn=<SelectBackward>) tensor(255)
n02105162_malinois 		 tensor(11.0313, grad_fn=<SelectBackward>) tensor(225)
n02091467_Norwegian_elkhound,_elkhound 		 tensor(9.7304, grad_fn=<SelectBackward>) tensor(174)
n02090721_Irish_wolfhound 		 tensor(8.9736, grad_fn=<SelectBackward>) tensor(170)
n02105056_groenendael 		 tensor(8.8621, grad_fn=<SelectBackward>) tensor(224)
n02116738_African_hunting_dog,_hyena_dog,_Cape_hunting_dog,_Lycaon_pictus 		 tensor(8.5262, grad_fn=<SelectBackward>) tensor(275)
n02088094_Afghan_hound,_Afghan 		 tensor(8.4578, grad_fn=<SelectBackward>) tensor(160)
n02090622_borzoi,_Russian_wolfhound 		 tensor(7.9833, grad_fn=<SelectBackward>) tensor(169)
n02105412_kelpie 		 tensor(7.8347, grad_fn=<SelectBackward>) tensor(227)


# Convert PyTorch to ONNX

In [12]:
!pip3 install onnxruntime
!pip3 install onnxruntime-gpu

  Using cached https://files.pythonhosted.org/packages/e6/97/39c630134268a29a7c26f5f1c8fd2f7ff089ccee567cb076087ddf1cb6e0/onnxruntime-0.1.4-cp35-cp35m-manylinux1_x86_64.whl
You are using pip version 8.1.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/17/cb/0def5a44db45c6d38d95387f20057905ce2dd4fad35c0d43ee4b1cebbb19/onnxruntime_gpu-0.1.3-cp35-cp35m-manylinux1_x86_64.whl
You are using pip version 8.1.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
torch.onnx._export(net, x, 'test_model.onnx', export_params=True)

tensor([[ 8.9698e-01, -3.0496e+00, -2.7042e+00, -7.4797e-01, -3.5622e+00,
         -1.7981e+00, -4.8486e+00,  1.0940e-01,  8.6848e-01,  5.3563e-02,
         -1.7047e+00, -3.4052e+00, -4.5000e+00, -3.7265e+00, -1.7852e+00,
         -2.2921e+00, -7.1484e-01, -1.4667e-01, -1.3266e+00, -4.1419e+00,
         -2.5389e+00, -7.5094e-01, -3.6083e+00,  1.3095e+00, -2.0261e+00,
          1.2687e+00, -1.3761e+00, -1.7112e+00,  6.9341e-01, -7.8196e-01,
         -3.0654e+00, -2.9808e+00, -1.4203e+00, -1.3106e+00,  3.2277e-02,
         -7.9055e-01,  1.7591e-01, -6.6472e-01, -1.2663e+00, -2.1434e+00,
         -1.4755e+00, -2.3319e+00, -2.3057e+00, -4.7204e-01, -1.6553e-01,
          9.6035e-01, -2.6927e+00, -1.9930e+00, -1.3999e+00,  4.2229e-02,
          1.6009e+00,  1.2011e+00, -4.7398e-01, -2.5052e+00, -1.0870e+00,
         -1.7356e+00, -3.8899e+00, -2.6196e+00, -2.6320e+00, -5.5144e-02,
          2.0895e-01, -8.4870e-01,  7.3489e-01,  6.4995e-01, -1.0003e+00,
         -1.5075e+00, -1.2006e+00, -3.

In [14]:
!ls -l test_model.onnx 

-rw-rw-r-- 1 ngeorgis ngeorgis 103242180 Jan 26 13:17 test_model.onnx


# Install the Intel OpenVino SDK

https://software.intel.com/en-us/openvino-toolkit